In [1]:
#CAT IN OPENING MOUTH
import cv2
import numpy as np
import dlib
from math import hypot
from scipy.spatial import distance as dist
from cvzone.SelfiSegmentationModule import SelfiSegmentation
# Loading Camera and Nose image and Creating mask
cap = cv2.VideoCapture(0)
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
BG = cv2.imread("catbg.jpg",cv2.IMREAD_COLOR)
background = cv2.resize(BG,(640,480))
BG2 = BG = cv2.imread("bg.jpg",cv2.IMREAD_COLOR)
background2 = cv2.resize(BG2,(640,480))
segmentor = SelfiSegmentation()
cloud_image = cv2.imread("cloud-01-removebg-preview.png",cv2.IMREAD_COLOR)
cat_image = cv2.imread("cat-removebg-preview.png",cv2.IMREAD_COLOR)
_, frame = cap.read()
rows, cols, _ = frame.shape
nose_mask = np.zeros((rows, cols), np.uint8)
# Loading Face detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [2]:
while True:
    _, frame = cap.read()
    nose_mask.fill(0)
    #lip_mask.fill(0)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(frame)
    segmentated_img = segmentor.removeBG(frame,background, threshold=0.9)
    for face in faces:
        landmarks = predictor(gray_frame, face)
        #MOUTH IS OPENING
        left_lip = (landmarks.part(48).x, landmarks.part(48).y)
        right_lip = (landmarks.part(54).x, landmarks.part(54).y)
        center_lip = (landmarks.part(66).x, landmarks.part(66).y)
        lips_width = int(hypot(left_lip[0] - right_lip[0],
                           left_lip[1] - right_lip[1]) * 1.2)
        lips_height = int(lips_width * 0.9)
        lip49 = (landmarks.part(49).x, landmarks.part(49).y)
        lip59 = (landmarks.part(59).x, landmarks.part(59).y)
        A = int(hypot(lip49[0] - lip59[0],
                           lip49[1] - lip59[1]) * 1.7)
        lip53 = (landmarks.part(53).x, landmarks.part(53).y)
        lip55 = (landmarks.part(55).x, landmarks.part(55).y)
        B = int(hypot(lip53[0] - lip55[0],
                           lip53[1] - lip55[1]) * 1.7)
        ratio = (A+B)/(2.0*lips_width)
        #segmentated_img = segmentor.removeBG(frame,background, threshold=0.9)
        if ratio > 0.5 :
            # cat position
            top_leftlip = (int(center_lip[0] - lips_width / 2),
                                int(center_lip[1] - lips_height / 2))
            bottom_rightlip = (int(center_lip[0] + lips_width / 2),
                        int(center_lip[1] + lips_height / 2))
            # Adding the cat
            lip_im = cv2.resize(cat_image, (lips_width,lips_height))
            lip_im_gray = cv2.cvtColor(lip_im, cv2.COLOR_BGR2GRAY)
            _, lip_mask = cv2.threshold(lip_im_gray, 25, 255, cv2.THRESH_BINARY_INV)
            lip_area = frame[top_leftlip[1]: top_leftlip[1] + lips_height,
                        top_leftlip[0]: top_leftlip[0] + lips_width]
            lip_area_no_lip = cv2.bitwise_and(lip_area, lip_area, mask=lip_mask)
            final_lip = cv2.add(lip_area_no_lip, lip_im)
            frame[top_leftlip[1]: top_leftlip[1] + lips_height,
                        top_leftlip[0]: top_leftlip[0] + lips_width] = final_lip
            segmentated_img = segmentor.removeBG(frame,background2, threshold=0.9)
    #segmentated_img = segmentor.removeBG(frame,background, threshold=0.9)
    cv2.imshow("Frame",segmentated_img)
    #cv2.imshow("lip_area",lip_area)
    #cv2.imshow("lip_area_no_lip",lip_area_no_lip)
    #cv2.imshow("final_lip",lip_mask)
    key = cv2.waitKey(1)
    if key == 27:
        break